In [8]:
import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t_spin
from pytenet.fermi_ops import generate_fermi_operators
import numpy as np
from scipy import sparse
import copy
import pytenet as ptn
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt

In [9]:
#load integrals
with h5py.File("data_water/eri_water.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("data_water/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)

#load Hamiltonian generated by above coefficients
H_correct = scipy.io.mmread('data_water/H_water_correct.mtx').tocsr()
e1, v1 = sparse.linalg.eigsh(H_correct, which = 'SA', k = 1)
e_ground = e1

7
(7, 28)
(28, 28)
rl errV: 2.8386751875274264e-12
abs errV: 2.0615721155266396e-11
errt: 7.097049412242525e-13
errh: 2.585427402664151e-13
errht: 9.079449636842276e-14


In [10]:
#generate thc_mpo
t_spin = get_t_spin(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t_spin)

In [11]:
hartree_state = generate_single_state(14, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [12]:
state_to_be_compressed = ptn.operation.apply_operator(H_mu_nu_list_spin_layer[0][0], hartree_state)
#assert(norm((H_mu_nu_list_spin_layer[0][0].as_matrix())@ (hartree_state.as_vector()) -  state_to_be_compressed.as_vector()) == 0)
state_to_be_compressed.bond_dims

[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]

In [13]:
#state_to_be_compressed.compress(tol=1e-10, mode='left')

In [23]:
copy_state = copy.deepcopy(state_to_be_compressed)
copy_state.compress_no_normalization_max_bond(tol=1e-4, mode='left', max_bond = 1)
print(copy_state.bond_dims)
print(norm(copy_state.as_vector() - state_to_be_compressed.as_vector()))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
3.42714933965464
